In [142]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [86]:
path = os.getcwd() + "/atribuna/milarquivos"
content_file_array = []
classifier_array = []

## Construção do DataFrame

In [87]:
def read_text(path, file_Name):
    with open(path, encoding="latin-1") as file:
        content_file = file.read()
        Build_DataFrame(content_file, file_Name)

In [88]:
def GetClassifier(file_Name):
    name_split =  file_Name.split(".")
    first_name = name_split[0];
    return first_name[(len(first_name) - 3) : len(first_name)]

In [89]:
def Build_DataFrame(content_file, file_Name):
    classifier =  GetClassifier(file_Name) 
    if(len(classifier) >= 3):        
        classifier_array.append(classifier)
        content_file_array.append(content_file)  

In [90]:
for file_Name in tqdm(os.listdir(path), desc="Carregar arquivos"):    
    if file_Name.endswith(".txt"):
        file_path = f"{path}/{file_Name}"
      
        # call read text file function
        read_text(file_path, file_Name)

Carregar arquivos: 100%|███████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 1153.08it/s]


In [91]:
textDataDrame =  pd.DataFrame({"Text":content_file_array, "Class":classifier_array})

In [92]:
textDataDrame.head(10)

,Text,Class
0,Os aprovados (edital 002/2006)\n<b> </b><br>\...,con
1,Agência de energia admite novo apagão\n<b> </...,eco
2,FGV garante que não há perigo \n<b> </b><br>\...,eco
3,PAINEL S/A\n<b> </b><br>\n <br>\nInvestidor p...,eco
4,Resultado do concurso da Polícia Civil \n<b> ...,con
5,Consumidor melhora de vida\n<b> As famílias\nc...,eco
6,NOTAS\n<b> </b><br>\n <br>\nPetrobras acusada...,eco
7,Pane afeta rede de cabos\n<b> </b><br>\n <br>...,eco
8,NOTAS\n<b> </b><br>\n <br>\nJustiça faz leilã...,eco
9,Blix desmente o governo americano \n<b> </b><...,int


# StopWords removing

In [93]:
#!conda install unidecode

import nltk
import unidecode
from bs4 import BeautifulSoup

# nltk.download()
#- averaged_perceptron_tagger
# - floresta
# - mac_morpho
# - machado
# - punkt
# - stopwords
# - wordnet
# - words

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [94]:
def addStopWordsToList(texto):
    example_sent = texto
    word_tokens = word_tokenize(example_sent)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = ""

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence += w + " "
    return filtered_sentence.strip()

In [95]:
stop_words = set(stopwords.words('portuguese'))
def returnDataFrameStopWords(dataFrame):
    dataReturn = pd.DataFrame()
    for row in tqdm(dataFrame.itertuples(index=True, name='Pandas'), desc="Limpeza de texto"):
        Text = row.Text
        Class = row.Class

        Text = BeautifulSoup(Text, "lxml").text
        Text = unidecode.unidecode(Text)
        Text = addStopWordsToList(Text)
        Text = Text.replace(" . "," ")
        Text = Text.replace(" , "," ")
        Text = Text.replace(" - "," ")
        Text = Text.replace(" : "," ")
        Text = Text.lower()

        linha = pd.Series([Text, Class])
        linha = pd.DataFrame([linha])
        dataReturn = pd.concat([linha, dataReturn], ignore_index=True)

    return dataReturn

In [96]:
df_text_clean = returnDataFrameStopWords(textDataDrame)

Limpeza de texto: 198it [00:00, 207.31it/s]


## Fatores de Ponderação

In [97]:
df_text_clean = df_text_clean.rename(columns = {0: 'Text', 1: 'Class'}, inplace = False)
df_text_clean.head()

,Text,Class
0,moqueca faz hartung cancelar agenda o governad...,pot
1,servidores fazem novo protesto mais vez servid...,pot
2,antecipada producao petrobras o presidente pet...,eco
3,governo faz cadastro sindicatos sao paulo o mi...,eco
4,panorama economico ano inclusao a coluna inaug...,eco


## Importando libs do scikit-learn

In [322]:
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf
from sklearn.feature_extraction.text import CountVectorizer # tf

## Criando a lista de classes

In [99]:
 classes =  ["at2","sro","inf","imo","ept","esp","fam","tvt","cid","mul","pot","tav","mic","opi","poc","reg","eco","bro","cit","con","int"]

## TF

In [100]:
vectorizer = CountVectorizer()
TF = vectorizer.fit_transform(df_text_clean.Text)

## TF-IDF

In [323]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_text_clean.Text)
df_tfidf = X.toarray()

# Lista de palavras

In [103]:
lista_palavras = []
for row in tqdm(df_text_clean.itertuples(index=True, name='Pandas')):
    palavras = row.Text.split(" ")
    for palavra in palavras:
        if(not len(palavra) <= 2):
            if(palavra not in lista_palavras):
                lista_palavras.append(palavra)

198it [00:01, 114.02it/s]


---

# frequencia palavra por classe

In [104]:
dic_doc_class = {}

for classe in tqdm(classes):
    lista_doc = []
    for row in df_text_clean.itertuples(index=True, name='Pandas'):  
        if (str(classe) == row.Class):
            lista_doc.append(row.Text)
    dic_doc_class[f'{classe}'] = lista_doc

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1614.97it/s]


In [105]:
from collections import Counter

In [106]:
df_relacao_class = pd.DataFrame()
df_relacao_class['Index'] = lista_palavras
for key in tqdm(dic_doc_class,"classes"):
    lista_docs =  dic_doc_class[key]
    docs = ""
    for doc in lista_docs:
        docs += f' {doc}'
        
    lista_quant =[]
    count = Counter(docs.split(" "))
    for termo in lista_palavras:
        quant = count[termo]
        lista_quant.append(quant)
        
    df_relacao_class[f'{key}'] = lista_quant
    
    
df_relacao_class = df_relacao_class.set_index('Index')

classes: 100%|████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 146.76it/s]


In [107]:
df_relacao_class.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
moqueca,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
faz,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,17,0,0,0,0
hartung,0,0,0,0,0,0,0,0,0,0,...,0,0,3,1,0,2,0,0,0,0
cancelar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
agenda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,0,0,0,0


# ni - o numero  de documentos em que um determinado termo ki aparece

In [108]:
dic_ni = {}

for termo in tqdm(lista_palavras):
    quant = 0 
    for row in df_text_clean.itertuples(index=True, name='Pandas'):
        doc = row.Text
        if(termo in doc):
            quant += 1
    dic_ni[f'{termo}'] = quant

100%|██████████████████████████████████████████████████████████████████████████| 13356/13356 [00:11<00:00, 1137.86it/s]


# a - quantidade de documentos que pertence a uma classe e posuem o termo ki

#  c - o numero  de documentos que pertencem a Cp, mas não possuem o termo ki 

In [109]:
df_a = pd.DataFrame()
df_a['Index'] = lista_palavras

df_c = pd.DataFrame()
df_c['Index'] = lista_palavras

for key in tqdm(dic_doc_class):
    lista_docs =  dic_doc_class[key]
    lista_quant_a =[]
    lista_quant_c =[]

    for termo in lista_palavras:
        quant_a = 0
        quant_c = 0
        
        for doc in lista_docs:
            if(termo in doc):
                quant_a += 1
            else:
                quant_c += 1
                
        lista_quant_a.append(quant_a)
        lista_quant_c.append(quant_c)
        
    df_a[f'{key}'] = lista_quant_a
    df_c[f'{key}'] = lista_quant_c
    
    
df_a = df_a.set_index('Index')
df_c = df_c.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:04<00:00,  4.98it/s]


# df_a

In [110]:
df_a.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
moqueca,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
faz,0,0,0,0,6,0,0,0,0,0,...,0,0,1,2,0,46,0,0,0,0
hartung,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,2,0,0,0,0
cancelar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
agenda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7,0,0,0,0


# df_c

In [111]:
df_c.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
moqueca,0,0,0,0,18,2,0,0,0,0,...,0,0,3,2,0,92,0,0,3,2
faz,0,0,0,0,12,2,0,0,0,0,...,0,0,2,0,0,46,0,0,3,2
hartung,0,0,0,0,18,2,0,0,0,0,...,0,0,2,1,0,90,0,0,3,2
cancelar,0,0,0,0,18,2,0,0,0,0,...,0,0,3,2,0,92,0,0,3,2
agenda,0,0,0,0,18,2,0,0,0,0,...,0,0,3,2,0,85,0,0,3,2


# b - a quantidade de documentos que possuem o termo ki, mas não pertence a Cp

# d o numero de documentos que não possuem ki e não pertencem a Cp.


In [112]:
df_b = pd.DataFrame()
df_b['Index'] = lista_palavras

df_d = pd.DataFrame()
df_d['Index'] = lista_palavras

for key in tqdm(dic_doc_class):
    lista_quant_b =[]
    lista_quant_d =[]
    
    for termo in lista_palavras:
        quant_b = 0
        quant_d = 0
        for classe in classes:
            if(classe != key):
                lista_docs =  dic_doc_class[classe]
                
                for doc in lista_docs:
                    if(termo in doc):
                        quant_b += 1
                    else:
                        quant_d += 1
                        
        lista_quant_b.append(quant_b)
        lista_quant_d.append(quant_d)
        
    df_b[f'{key}'] = lista_quant_b
    df_d[f'{key}'] = lista_quant_d
    
df_b = df_b.set_index('Index')
df_d = df_d.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [01:21<00:00,  3.89s/it]


# df_b

In [113]:
df_b.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
moqueca,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
faz,91,91,91,91,85,91,91,91,91,91,...,91,91,90,89,91,45,91,91,91,91
hartung,24,24,24,24,24,24,24,24,24,24,...,24,24,23,23,24,22,24,24,24,24
cancelar,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
agenda,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,12,19,19,19,19


# df_d

In [114]:
df_d.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
moqueca,196,196,196,196,178,194,196,196,196,196,...,196,196,193,194,196,104,196,196,193,194
faz,107,107,107,107,95,105,107,107,107,107,...,107,107,105,107,107,61,107,107,104,105
hartung,174,174,174,174,156,172,174,174,174,174,...,174,174,172,173,174,84,174,174,171,172
cancelar,196,196,196,196,178,194,196,196,196,196,...,196,196,193,194,196,104,196,196,193,194
agenda,179,179,179,179,161,177,179,179,179,179,...,179,179,176,177,179,94,179,179,176,177


---

## Conta quantas classes o termo ki aparece

In [115]:
def get_row_class(row, classe):
    dic_classe = {
        'at2':row.at2,
        'sro':row.sro,
        'inf':row.inf,
        'imo':row.imo,
        'ept':row.ept,
        'esp':row.esp,
        'fam':row.fam,
        'tvt':row.tvt,
        'cid':row.cid,
        'mul':row.mul,
        'pot':row.pot,
        'tav':row.tav,
        'mic':row.mic,
        'opi':row.opi,
        'poc':row.poc,
        'reg':row.reg,
        'eco':row.eco,
        'bro':row.bro,
        'cit':row.cit,
        'con':row.con,
        'int':row.int
    }
    return dic_classe[f'{classe}']
    

In [116]:
def num_ki_em_categoria():

    dic_num_ki_em_categoria ={}
    for row in tqdm(df_relacao_class.itertuples(index=True, name='Pandas')):
        ci = 0
        for classe in classes:
            if get_row_class(row,classe)!= 0:
                ci += 1
        dic_num_ki_em_categoria[f'{row.Index}'] = ci
        
    return dic_num_ki_em_categoria

# Bag of Word

In [117]:
bag_of_word = {}
bag_of_word['a'] = df_a
bag_of_word['b'] = df_b
bag_of_word['c'] = df_c
bag_of_word['d'] = df_d
bag_of_word['freq_pal_class'] = df_relacao_class
bag_of_word['ci'] = num_ki_em_categoria()
bag_of_word['ni'] = dic_ni

13356it [00:00, 36756.94it/s]


## ICF

In [118]:
import math 

In [119]:
def calculeteICF():
    dic_num_ki_em_categoria = bag_of_word['ci']
    icfs = {}
    C = len(classes)
    for key in dic_num_ki_em_categoria:
        Ci = dic_num_ki_em_categoria[key]
        valor = math.log((1 + (C/Ci)))
        icfs[f'{key}'] = valor
    return icfs

In [120]:
icfs = calculeteICF() 

In [121]:
df_icf = pd.DataFrame()
df_icf['Index'] = df_text_clean.index

for termo in tqdm(lista_palavras):
    
    list_icfs = []
    
    for row in df_text_clean.itertuples(index=True, name='Pandas'):
        doc = row.Text
        if termo in doc:
            list_icfs.append(icfs[termo])
        else:
            list_icfs.append(0)
    
    df_icf[f'{termo}'] = list_icfs
    
df_icf = df_icf.set_index('Index')

100%|███████████████████████████████████████████████████████████████████████████| 13356/13356 [00:23<00:00, 560.10it/s]


In [122]:
df_icf.head()

,moqueca,faz,hartung,cancelar,agenda,governador,reacao,alergica,apos,comer,...,laurivan,pelissari,luhanna,"75,0",marcal,rozeli,walleska,zulina,fairich,29/08/2006
Index,,,,,,,,,,,,,,,,,,,,,
0,3.091042,2.079442,1.832581,3.091042,2.442347,1.648659,1.832581,3.091042,1.832581,2.442347,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,2.079442,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,2.079442,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.442347,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

## Calculando IDF

In [123]:
idfDict = {}
N = len(df_text_clean)

for termo, value in tqdm(bag_of_word['ni'].items()):
    idfDict[termo] = math.log10(N / (float(value)))

100%|███████████████████████████████████████████████████████████████████████| 13356/13356 [00:00<00:00, 1545696.27it/s]


# Calculo qui, rf, icf based, rf*idf

In [124]:
df_qui_quadrado = pd.DataFrame()
df_qui_quadrado['Index'] = lista_palavras

df_rf = pd.DataFrame()
df_rf['Index'] = lista_palavras

df_icf_based = pd.DataFrame()
df_icf_based['Index'] = lista_palavras

C = len(classes)
dic_num_ki_em_categoria = bag_of_word['ci']

df_rfIdf = pd.DataFrame()
df_rfIdf['Index'] = lista_palavras
    

for classe in tqdm(classes):
    
    lista_valores_qui = []
    lista_valores_rf = []
    lista_valores_icf_based = []
    lista_valores_rfidf = []
    
    for termo in lista_palavras:
        a = bag_of_word['a'].loc[termo, classe]
        b = bag_of_word['b'].loc[termo, classe]
        c = bag_of_word['c'].loc[termo, classe]
        d = bag_of_word['d'].loc[termo, classe]
        
        # qui
        numerador = (len(df_text_clean) * math.pow(((a * d) - (c * d)), 2))
        denominador = ((a + c) * (b + d) * (a + b) * (c + d))

        if (numerador == 0):
            value_qui = 0
        else:
            value_qui = numerador/denominador
            
        # RF
        value_rf =  math.log(2 + (a / (max(1,b))))
        
        #ICF-based
        Ci = dic_num_ki_em_categoria[termo]
        valor_icf_based = math.log((2 + (a/max(1,b)) * (C/Ci)))
        
        #RF*IDF
        valueIdf =  idfDict[termo]
        value_rfidf =  value_rf * valueIdf
        
        #adicionando a listas
        lista_valores_qui.append(value_qui)
        lista_valores_rf.append(value_rf)
        lista_valores_icf_based.append(valor_icf_based)
        lista_valores_rfidf.append(value_rfidf)

    df_qui_quadrado[f'{classe}'] = lista_valores_qui
    df_rf[f'{classe}'] = lista_valores_rf
    df_icf_based[f'{classe}'] = lista_valores_icf_based
    df_rfIdf[f'{classe}'] = lista_valores_rfidf

df_qui_quadrado = df_qui_quadrado.set_index('Index')
df_rf = df_rf.set_index('Index')
df_icf_based = df_icf_based.set_index('Index')
df_rfIdf = df_rfIdf.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:09<00:00,  2.29it/s]


## Construção dataframe documentos x termos 

In [125]:
df_doc_rf= pd.DataFrame()
df_doc_rf['Index'] = df_text_clean.index

df_doc_qui= pd.DataFrame()
df_doc_qui['Index'] = df_text_clean.index

df_doc_icf_based= pd.DataFrame()
df_doc_icf_based['Index'] = df_text_clean.index

df_doc_rfIdf= pd.DataFrame()
df_doc_rfIdf['Index'] = df_text_clean.index

for termo in tqdm(lista_palavras):
    
    list_rf = []
    list_qui = []
    list_icf_based = []
    list_rfIdf = []
    
    
    for row in df_text_clean.itertuples(index=True, name='Pandas'):
        doc = row.Text
        classe = row.Class
        
        if termo in doc:
            value_rf = df_rf.loc[termo,classe]
            value_qui = df_qui_quadrado.loc[termo,classe]
            value_icf_based = df_icf_based.loc[termo,classe]
            value_rfIdf = df_rfIdf.loc[termo,classe]
            
            list_rf.append(value_rf)
            list_qui.append(value_qui)
            list_icf_based.append(value_icf_based)
            list_rfIdf.append(value_rfIdf)
        else:
            list_rf.append(0)
            list_qui.append(0)
            list_icf_based.append(0)
            list_rfIdf.append(0)
    
    df_doc_rf[f'{termo}'] = list_rf
    df_doc_qui[f'{termo}'] = list_qui
    df_doc_icf_based[f'{termo}'] = list_icf_based
    df_doc_rfIdf[f'{termo}'] = list_rfIdf
    
    
df_doc_rf = df_doc_rf.set_index('Index')
df_doc_qui= df_doc_qui.set_index('Index')
df_doc_icf_based = df_doc_icf_based.set_index('Index')
df_doc_rfIdf = df_doc_rfIdf.set_index('Index')

100%|███████████████████████████████████████████████████████████████████████████| 13356/13356 [00:58<00:00, 227.80it/s]


---

# Machine Learning

In [287]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score, roc_curve,classification_report,r2_score
from sklearn import model_selection as ms
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.neighbors import NearestCentroid

In [305]:
def metricas(acuracia, precision,recal, micro_f1, macro_f1):
    print(f'Acurácia: %.2f' % (acuracia*100))
    print(f'precision: %.2f' % (precision*100))
    print(f'recal: %.2f' % (recal*100))
    print(f'micro-f1: %.2f' % (micro_f1*100))
    print(f'macro-f1: %.2f' % (macro_f1*100))

In [269]:
dic_dfs = {}
dic_dfs["TF"] = TF
dic_dfs["TF-IDF"] = TF
dic_dfs["ICF"] = df_icf
dic_dfs["X²"] = df_doc_qui
dic_dfs["ICF-based"] = df_doc_icf_based
dic_dfs["RF"] = df_doc_rf
dic_dfs["rfXidf"] = df_doc_rfIdf

# KNN

### data frame de resultados

In [312]:
data_results_knn = pd.DataFrame()

In [313]:
data_results_knn['Index'] = dic_dfs.keys()
lista_acuracia =[]
lista_precision =[]
lista_recal =[]
lista_mif1 =[]
lista_maf1 =[]

for key in dic_dfs:
    # features
    X = dic_dfs[key]

    #response
    Y = df_text_clean.Class
    
    #treino e teste
    x_train, x_test, y_train, y_test = ms.train_test_split(X,Y,test_size=0.33, random_state=5)
    
    #model description
    model_knn = KNeighborsClassifier(n_neighbors=1)
    
    #train
    model_knn.fit(x_train, y_train)
    
    #pred
    pred = model_knn.predict(x_test)
    
    
    acuracia = metrics.accuracy_score(y_test, pred)
    lista_acuracia.append(acuracia)
    
    precision = precision_score(y_test, pred, average='weighted')
    lista_precision.append(precision)
    
    recal = recall_score(y_test, pred, average='weighted')
    lista_recal.append(recal)
    
    micro_f1 = f1_score(y_test, pred, average='micro')
    lista_mif1.append(micro_f1)
    
    macro_f1 = f1_score(y_test, pred, average='macro')
    lista_maf1.append(macro_f1)
    
    print(f"********** PONDERAÇÂO {key}")
    metricas(acuracia, precision, recal, micro_f1, macro_f1)
    
data_results_knn['acuracia'] = lista_acuracia
data_results_knn['precision'] = lista_precision
data_results_knn['recal'] = lista_recal
data_results_knn['micro-f1'] = lista_mif1
data_results_knn['macro-f1'] = lista_maf1
    
data_results_knn = data_results_knn.set_index('Index')

********** PONDERAÇÂO TF
Acurácia: 57.58
precision: 76.90
recal: 57.58
micro-f1: 57.58
macro-f1: 44.48
********** PONDERAÇÂO ICF
Acurácia: 57.58
precision: 71.35
recal: 57.58
micro-f1: 57.58
macro-f1: 24.86
********** PONDERAÇÂO X²
Acurácia: 1.52
precision: 0.76
recal: 1.52
micro-f1: 1.52
macro-f1: 8.33
********** PONDERAÇÂO ICF-based
Acurácia: 19.70
precision: 37.91
recal: 19.70
micro-f1: 19.70
macro-f1: 18.72
********** PONDERAÇÂO RF
Acurácia: 36.36
precision: 91.75
recal: 36.36
micro-f1: 36.36
macro-f1: 27.91
********** PONDERAÇÂO rfXidf
Acurácia: 40.91
precision: 64.49
recal: 40.91
micro-f1: 40.91
macro-f1: 10.64


# Metricas KNN para cada ponderação

In [314]:
data_results_knn

,acuracia,precision,recal,micro-f1,macro-f1
Index,,,,,
TF,0.575758,0.769011,0.575758,0.575758,0.444780
ICF,0.575758,0.713455,0.575758,0.575758,0.248594
X²,0.015152,0.007576,0.015152,0.015152,0.083333
ICF-based,0.196970,0.379091,0.196970,0.196970,0.187244
RF,0.363636,0.917460,0.363636,0.363636,0.279112
rfXidf,0.409091,0.644886,0.409091,0.409091,0.106397


# Centroid

### data frame de resultados

In [317]:
data_results_centroid = pd.DataFrame()

In [318]:
data_results_centroid['Index'] = dic_dfs.keys()
lista_acuracia =[]
lista_precision =[]
lista_recal =[]
lista_mif1 =[]
lista_maf1 =[]

for key in dic_dfs:
    # features
    X = dic_dfs[key]

    #response
    Y = df_text_clean.Class
    
    #treino e teste
    x_train, x_test, y_train, y_test = ms.train_test_split(X,Y,test_size=0.33, random_state=5)
    
    #model description
    model_centroid = NearestCentroid()
    
    #train
    model_centroid.fit(x_train, y_train)
    
    #pred
    pred = model_centroid.predict(x_test)
    
    acuracia = metrics.accuracy_score(y_test, pred)
    lista_acuracia.append(acuracia)
    
    precision = precision_score(y_test, pred, average='weighted')
    lista_precision.append(precision)
    
    recal = recall_score(y_test, pred, average='weighted')
    lista_recal.append(recal)
    
    micro_f1 = f1_score(y_test, pred, average='micro')
    lista_mif1.append(micro_f1)
    
    macro_f1 = f1_score(y_test, pred, average='macro')
    lista_maf1.append(macro_f1)
    
    print(f"********** PONDERAÇÂO {key}")
    metricas(acuracia, precision, recal, micro_f1, macro_f1)
    
data_results_centroid['acuracia'] = lista_acuracia
data_results_centroid['precision'] = lista_precision
data_results_centroid['recal'] = lista_recal
data_results_centroid['micro-f1'] = lista_mif1
data_results_centroid['macro-f1'] = lista_maf1
    
data_results_centroid = data_results_centroid.set_index('Index')

********** PONDERAÇÂO TF
Acurácia: 92.42
precision: 90.10
recal: 92.42
micro-f1: 92.42
macro-f1: 64.40
********** PONDERAÇÂO ICF
Acurácia: 80.30
precision: 80.91
recal: 80.30
micro-f1: 80.30
macro-f1: 43.54
********** PONDERAÇÂO X²
Acurácia: 1.52
precision: 0.51
recal: 1.52
micro-f1: 1.52
macro-f1: 6.25
********** PONDERAÇÂO ICF-based
Acurácia: 86.36
precision: 89.74
recal: 86.36
micro-f1: 86.36
macro-f1: 57.14
********** PONDERAÇÂO RF
Acurácia: 90.91
precision: 90.47
recal: 90.91
micro-f1: 90.91
macro-f1: 60.22
********** PONDERAÇÂO rfXidf
Acurácia: 86.36
precision: 86.84
recal: 86.36
micro-f1: 86.36
macro-f1: 62.16


# Metricas Centroid para cada ponderação

In [319]:
data_results_centroid

,acuracia,precision,recal,micro-f1,macro-f1
Index,,,,,
TF,0.924242,0.901010,0.924242,0.924242,0.644009
ICF,0.803030,0.809063,0.803030,0.803030,0.435417
X²,0.015152,0.005051,0.015152,0.015152,0.062500
ICF-based,0.863636,0.897367,0.863636,0.863636,0.571399
RF,0.909091,0.904718,0.909091,0.909091,0.602242
rfXidf,0.863636,0.868398,0.863636,0.863636,0.621609
